In [1306]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [1307]:
RANDOM_STATE = 42

In [1308]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target
X

/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include th

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [1309]:
learn_df, test_df, learn_y, test_y = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

print('Size of learn df: '+ str(len(learn_df))+ ', size of test df: '+str(len(test_df)))
print('Size of learn_y: '+str(len(learn_y)) + ', size of test y: '+str(len(test_y)))

Size of learn df: 404, size of test df: 102
Size of learn_y: 404, size of test y: 102


2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [1310]:
reg = LinearRegression().fit(learn_df, learn_y)
predicted_y = reg.predict(test_df)
r2_score(test_y, predicted_y)

0.6687594935356366

In [1311]:
ridge = Ridge()
ridge.fit(learn_df, learn_y)
predicted_ridge_y = ridge.predict(test_df)
r2_score(test_y, predicted_ridge_y)

0.666222167016852

In [1312]:
lasso = Lasso()
lasso.fit(learn_df, learn_y)
predicted_lasso_y = lasso.predict(test_df)
r2_score(test_y,predicted_lasso_y)

0.6671453631686304

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [1313]:
alpha_arr = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]
alpha_dict = {'alpha': alpha_arr}
kf = KFold(5)

In [1314]:
gcv = GridSearchCV(Ridge(random_state=RANDOM_STATE), alpha_dict, cv=kf)
gcv.fit(learn_df, learn_y)
gcv.get_params()
gcv_predicted = gcv.predict(test_df)
r2 = r2_score(test_y, gcv_predicted)
print('GridSearchCV + Ridge:  '+str(gcv.best_params_)+ ' r2_score = '+ str(r2)) 


GridSearchCV + Ridge:  {'alpha': 1e-05} r2_score = 0.6687594856409733


In [1315]:
rcv = RidgeCV(alphas=alpha_arr, cv=kf)
rcv.fit(learn_df, learn_y)
rcv_predicted = rcv.predict(test_df)
r2 = r2_score(test_y, rcv_predicted)
print('RidgeCV:  {\'alpha\':'+str(rcv.alpha_)+ '} r2_score = '+ str(r2)) 

RidgeCV:  {'alpha':1e-05} r2_score = 0.6687594856409733


In [1316]:
lcv = LassoCV(alphas = alpha_arr, random_state=RANDOM_STATE, selection='random')
lcv.fit(learn_df, learn_y)
predicted_lcv_y = lcv.predict(test_df)
r2 = r2_score(test_y, predicted_lcv_y)
print('LassoCV:  {\'alpha\':'+str(lcv.alpha_)+ '} r2_score = '+ str(r2)) 

LassoCV:  {'alpha':1e-05} r2_score = 0.6687598654554501


In [1317]:
glcv = GridSearchCV(Lasso(random_state=RANDOM_STATE, selection='random'), alpha_dict)
glcv.fit(learn_df, learn_y)
glcv_predicted = glcv.predict(test_df)
r2 = r2_score(test_y, glcv_predicted)
print('GridSearchCV + Lasso:  '+str(glcv.best_params_)+ ' r2_score = '+ str(r2)) 

GridSearchCV + Lasso:  {'alpha': 1e-05} r2_score = 0.6687598654554501


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1318]:
lasso_pipeline = make_pipeline(StandardScaler(), RidgeCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6659677905050334

In [1319]:
lasso_pipeline = make_pipeline(MinMaxScaler(), RidgeCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6700309977617867

In [1320]:
lasso_pipeline = make_pipeline(StandardScaler(), LassoCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6683883969336302

In [1321]:
lasso_pipeline = make_pipeline(MinMaxScaler(), LassoCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6688640223695405

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1322]:
lasso_pipeline = make_pipeline(StandardScaler(), RidgeCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6659677905050334

In [1323]:
lasso_pipeline = make_pipeline(MinMaxScaler(), RidgeCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6700309977617867

In [1324]:
lasso_pipeline = make_pipeline(StandardScaler(), LassoCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6687590383347171

In [1325]:
lasso_pipeline = make_pipeline(MinMaxScaler(), LassoCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

0.6687605073677362

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1326]:
pp = Pipeline(steps=[('scaler', StandardScaler()),('transform', PolynomialFeatures()), ('RidgeCV', RidgeCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)


0.8180465877242975

In [1327]:
pp = Pipeline(steps=[('scaler', MinMaxScaler()),('transform', PolynomialFeatures()), ('RidgeCV', RidgeCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)


0.8500630422288733

In [1328]:
pp = Pipeline(steps=[('scaler', StandardScaler()),('transform', PolynomialFeatures()), ('LassoCV', LassoCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)

/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0301893789678616, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gram(
/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.770304642251631, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gram(
/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5532441063974147, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gr

0.8165842609968705

In [1329]:
pp = Pipeline(steps=[('scaler', MinMaxScaler()),('transform', PolynomialFeatures()), ('LassoCV', LassoCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)

/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.31891165662455, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gram(
/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.158842619339339, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gram(
/Users/max/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.53233775802255, tolerance: 2.8821090464396284
  model = cd_fast.enet_coordinate_descent_gram(


0.8481225939905862

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [1330]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [1331]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.